In [1]:
import numpy as np
import pandas as pd
import time

# 01.112 Machine Learning Design Project

## About the Project

We have 4 datasets in the `/data` folder. For each dataset, there is: 
- a labelled training set train, 
- an unlabelled development set `dev.in`
- a labelled development set `dev.out` 

The labelled data has the format of: `token` `\t` `tag`
- one token per line
- token and tag separated by tab 
- single empty lines that separates sentences

For the labels, they are slightly different for different datasets.
- SG, CN (Entity):
    - B-*: Beginning of entity
    - I-*: Inside of entity
    - O: Outside of any entity
- EN, AL (Phrase):
    - B-VP: Beginning of Verb Phrase
    - I-VP: Inside of Verb Phrase
    - *-NP: Noun Phrase
    - *PP: Propositional Phrase
    - O: Outside of any phrase

*Goal*: Build sequence labelling systems from training data (x) and use it to predict tag sequences for new sentences (y).

## Team members 
- Andri Setiawan Susanto
- Eldon Lim 
- Tey Siew Wen

## Part 1
Already completed individually.

## Part 2

a) Write a function that estimates the emission parameters from the training set using MLE (maximum likelihood estimation):

In [2]:
def emissionPara(xy,y):
    e_x_y= xy/y
    
    return e_x_y

b)

1. Make a modified training set by replacing those words that appear $<k$ times in the training set with a special word token `#UNK#` before training.
2. During testing phase, ifaworddoesnot appear in the modified training set, we also replace that wordwith `#UNK#`.
3. Compute Emission Paramters with the function in (a)

In [3]:
k = 3
replaceWord = "#UNK#" 

In [4]:
def split_into_columns(df_column):
    new = df_column.str.split(" ", n=1, expand=True)
    return new[0], new[1]

In [5]:
# def preprocess_training(data,k):
#     global replaceWord
    
#     start = time.process_time()   
#     x_dic = {}
    
#     # dropping null value columns e.g. index_col to avoid errors 
#     df= pd.read_csv(data, sep='/n', delimiter=None, names=['original'],index_col=False, engine="python")
    
#     # new data frame with split value columns 
#     df["x"], df["y"] = split_into_columns(df["original"])

#     # df display: record x value and replace y values with replaceWord when necessary, in respective dictionaries
#     uniqueX, uniqueCountX= np.unique(df['x'].astype(str),return_counts=True)
#     for i in range(len(uniqueX)):
#         x_dic[uniqueX[i]] = uniqueCountX[i]

#     for i, text in enumerate(df['x']):
#         if x_dic[text] < k:
#             df['x'][i] = replaceWord
#             df['original'][i]=df['original'][i].replace(text,replaceWord, 1)
    
#     y_dic={}
    
#     uniqueY, uniqueCountY= np.unique(df['y'].astype(str),return_counts=True)
#     for i in range(len(uniqueY)):
#         y_dic[uniqueY[i]] = uniqueCountY[i]
        
#     xy_dic = {}
#     df1= df.copy()
    
#     # Get a tuple of unique values & their count from a numpy array
#     df1.dropna(inplace = True) 
#     uniqueXY, uniqueCountXY= np.unique(df1['original'].astype(str),return_counts=True)

#     for i in range(len(uniqueXY)):
#         xy_dic[uniqueXY[i]] = uniqueCountXY[i]
#     # print('Unique Values : ', uniqueValues)
    
#     # print('Count of Unique Values : ', uniqueCount)
#     dft = pd.DataFrame([uniqueXY,uniqueCountXY]).T
#     dft=dft.rename({0:'x_y',1:'count_x_y'},axis='columns')
    
#     dft['count_y']=0
#     for i,text in enumerate(dft['x_y']):
#         data = text.split(" ")
#         dft['count_y'][i]=y_dic[data[1]]
        
#     dft['emission']=emissionPara(dft['count_x_y'], dft['count_y'])
    
#     # new data frame with split value columns 
#     dft1 = dft.copy()
#     dft1["x"], dft1["y"] = split_into_columns(dft1["x_y"])
    
#     xy_pred_dic = {}

#     for word in dft1['x']:
#         index = pd.Series.idxmax((dft1.loc[dft1['x'] == word]['emission']).astype(float))
#         xy_pred_dic[word]=dft1['y'][index] 
#     print("Time taken for train data: ", time.process_time() - start)
#     return xy_pred_dic

In [6]:
def preprocess_training(data):
    global replaceWord
    
      
    x_dic = {}
    
    # dropping null value columns e.g. index_col to avoid errors 
    df= pd.read_csv(data, sep='/n', delimiter=None, names=['original'],index_col=False, engine="python")
    
    # new data frame with split value columns 
    df["x"], df["y"] = split_into_columns(df["original"])
    
    return df
   

In [7]:
def uniqueCount(df,k,replaceWord):
    start = time.process_time() 
    x_dic={}
     # df display: record x value and replace y values with replaceWord when necessary, in respective dictionaries
    uniqueX, uniqueCountX= np.unique(df['x'].astype(str),return_counts=True)
    for i in range(len(uniqueX)):
        x_dic[uniqueX[i]] = uniqueCountX[i]

    for i, text in enumerate(df['x']):
        if x_dic[text] < k:
            df['x'][i] = replaceWord
            df['original'][i]=df['original'][i].replace(text,replaceWord, 1)
    
    y_dic={}
    
    uniqueY, uniqueCountY= np.unique(df['y'].astype(str),return_counts=True)
    for i in range(len(uniqueY)):
        y_dic[uniqueY[i]] = uniqueCountY[i]
        
    xy_dic = {}
    df1= df.copy()
    
    # Get a tuple of unique values & their count from a numpy array
    df1.dropna(inplace = True) 
    uniqueXY, uniqueCountXY= np.unique(df1['original'].astype(str),return_counts=True)

    for i in range(len(uniqueXY)):
        xy_dic[uniqueXY[i]] = uniqueCountXY[i]
    # print('Unique Values : ', uniqueValues)
    
    # print('Count of Unique Values : ', uniqueCount)
    dft = pd.DataFrame([uniqueXY,uniqueCountXY]).T
    dft=dft.rename({0:'x_y',1:'count_x_y'},axis='columns')
    
    dft['count_y']=0
    for i,text in enumerate(dft['x_y']):
        data = text.split(" ")
        dft['count_y'][i]=y_dic[data[1]]
    print("Time taken for train data: ", time.process_time() - start)
    return dft

In [8]:
def emissionCalcu(df):
    emission_dict={}
    df['emission']=emissionPara(df['count_x_y'], df['count_y'])
    for i in range(df.shape[0]):
        emission_dict[df['x_y'][i]]= df['emission'][i]
    return df,emission_dict

def xyPrediction(dft):
    # new data frame with split value columns 
    dft1 = dft.copy()
    dft1["x"], dft1["y"] = split_into_columns(dft1["x_y"])
    
    xy_pred_dic = {}

    for word in dft1['x']:
        index = pd.Series.idxmax((dft1.loc[dft1['x'] == word]['emission']).astype(float))
        xy_pred_dic[word]=dft1['y'][index] 
    
    return xy_pred_dic

In [9]:
def preprocess_test(data,k):
    global replaceWord
    
    start = time.process_time()   

    testdf1= pd.read_csv(data, sep='/n', delimiter=None, names=['original'],index_col=False, engine="python")
    testdf= pd.read_csv(data, sep='/n', delimiter=None, names=['original'],index_col=False,skip_blank_lines=False, engine="python")

    x_dic = {}

    uniqueX, uniqueCountX= np.unique(testdf1['original'].astype(str),return_counts=True)
    for i in range(len(uniqueX)):
        x_dic[uniqueX[i]] = uniqueCountX[i]

    testdf['modified']=''
#     print(testdf)
    for i, text in enumerate(testdf['original']):
    #         df['x'][i] = replaceWord
        try:
            if text not in xy_pred_dic:
            
                testdf['modified'][i]=testdf['original'][i].replace(text,replaceWord)
            else:
                testdf['modified'][i]=testdf['original'][i]
        except:
            continue
    testdf['predict_label']=''
    for index, word in enumerate(testdf['modified']):
#     print(word)
        try:
            testdf['predict_label'][index]= xy_pred_dic[word]
        except:
            continue
    print("Time taken for test data: ",time.process_time() - start)
    return testdf

For all the four datasets EN, AL, CN, and SG, learn these parameters with `train`, and evaluate your
system on the development set `dev.in` for each of the dataset. Write your output to `dev.p2.out`
for the four datasets respectively. Compare your outputs and the gold-standard outputs in `dev.out`
and report the precision, recall and F scores of such a baseline system for each dataset.

In [10]:
data_folders = ["AL", "EN","CN","SG"]
for x in ["EN"]:
    print("Performing sentiment analysis for data folder ", x)
    train_data = "./data/{}/train".format(x)
    test_data = "./data/{}/dev.in".format(x)
    test_result = "./data/{}/dev.out".format(x)
    
    predata = preprocess_training(train_data)
    countData=uniqueCount(predata,k,replaceWord)
    emissiondf = emissionCalcu(countData)
    emission_dict = emissiondf[1]
    xy_pred_dic = xyPrediction(emissiondf[0])
    testdf = preprocess_test(test_data,k)
    
    testresultdf = pd.read_csv(test_result, sep='/n', delimiter=None, names=['original'],index_col=False, engine="python")
    new = testresultdf["original"].str.split(" ", n=1,expand=True) 

    # making separate first name column from new data frame 
    testresultdf["x"]= new[0] 

    # making separate last name column from new data frame 
    testresultdf["y"]= new[1]
    final = pd.DataFrame()
    
    final['result'] = testdf['modified'] + ' ' + testdf['predict_label']
    print(final.head(3))
    
    print("Writing the final result to dev.out...")
    f = open('./output/{}/dev.p2.out'.format(x) ,'w')
    for word in final['result']:
        f.write(word + '\n')
    f.close()
    

Performing sentiment analysis for data folder  EN


C:\Users\cooll\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Time taken for train data:  279.53125
Time taken for test data:  8.1875
         result
0      HBO B-NP
1      has B-VP
2  close B-ADJP
Writing the final result to dev.out...


## Part 3

Write a function that estimates the transition parameters from the training set using MLE (maximum likelihood estimation):

In [11]:
test = pd.read_csv("./data/EN/train", sep='/n', delimiter=None, names=['original'],index_col=False, engine="python", skip_blank_lines=False)
# test.replace(np.nan, None, inplace=True)
test.isnull().sum()

original    7663
dtype: int64

In [12]:
x, y = split_into_columns(test["original"])
y

0           B-NP
1           I-NP
2           B-VP
3         B-ADVP
4         B-ADJP
5         I-ADJP
6         I-ADJP
7           B-PP
8           B-NP
9           I-NP
10          B-PP
11          B-NP
12          I-NP
13             O
14             O
15        B-ADJP
16        I-ADJP
17        I-ADJP
18          B-PP
19          B-NP
20          B-VP
21          B-PP
22          B-NP
23          I-NP
24          I-NP
25             O
26           NaN
27          B-NP
28          B-VP
29        B-SBAR
           ...  
189261      I-NP
189262         O
189263      B-VP
189264      B-PP
189265      B-NP
189266      B-PP
189267      B-NP
189268      I-NP
189269      B-PP
189270      B-NP
189271         O
189272      B-VP
189273      B-PP
189274      B-NP
189275      I-NP
189276    B-SBAR
189277         O
189278      B-NP
189279      I-NP
189280      I-NP
189281      B-VP
189282      I-VP
189283      I-VP
189284      I-VP
189285      I-VP
189286      B-PP
189287      B-NP
189288      I-

In [13]:
from collections import Counter, defaultdict

def transitionPara(data):
    x, y = split_into_columns(test["original"])
    xy_dic = dict(zip(x, y))
    
    # Get bottom count (Count(yi))
    y_count = Counter(y)
    
    # Get top count (Count(yi-1, yi))
    subseq_count = defaultdict(int)
    for i in range(len(y)-1):    
        y1 = y[i]
        y2 = y[i+1]
        
        if i == 0:
            subseq_count[("START", y1)] +=1
            y_count["START"] +=1
        if pd.isna(y1):
            subseq_count[("START", y2)] +=1
            y_count["START"] +=1
        elif i == len(y)-1 or pd.isna(y2):
            subseq_count[(y1, "END")] +=1
            y_count["END"] +=1
        else:
            subseq_count[y1,y2] += 1
    
    # Calculation of transition params
    result = np.empty(len(y)+2)
    transition_dict = {}
    
    for k,v in subseq_count.items():
        y1 = k[0]
        y2 = k[1]
        transition_dict[y1,y2] = subseq_count[y1,y2] / y_count[y2]
     
    return transition_dict, subseq_count, y_count

transition_dic, subseq_count, y_count = transitionPara(test)

In [14]:
subseq_count

defaultdict(int,
            {('START', 'B-NP'): 4966,
             ('B-NP', 'I-NP'): 32390,
             ('I-NP', 'B-VP'): 7365,
             ('B-VP', 'B-ADVP'): 570,
             ('B-ADVP', 'B-ADJP'): 59,
             ('B-ADJP', 'I-ADJP'): 490,
             ('I-ADJP', 'I-ADJP'): 84,
             ('I-ADJP', 'B-PP'): 164,
             ('B-PP', 'B-NP'): 17064,
             ('I-NP', 'B-PP'): 8544,
             ('I-NP', 'O'): 12410,
             ('O', 'O'): 2710,
             ('O', 'B-ADJP'): 209,
             ('B-NP', 'B-VP'): 6164,
             ('B-VP', 'B-PP'): 1803,
             ('I-NP', 'I-NP'): 22201,
             ('O', 'END'): 7598,
             ('B-VP', 'B-SBAR'): 467,
             ('B-SBAR', 'B-NP'): 1657,
             ('B-VP', 'B-NP'): 6304,
             ('B-VP', 'O'): 1231,
             ('O', 'B-NP'): 8288,
             ('I-NP', 'B-NP'): 2601,
             ('B-ADVP', 'B-PP'): 608,
             ('B-VP', 'I-VP'): 6828,
             ('I-VP', 'B-NP'): 3610,
             ('O', 'B-VP

In [15]:
transition_dic

{('START', 'B-NP'): 0.10497833210020083,
 ('B-NP', 'I-NP'): 0.593321243428404,
 ('I-NP', 'B-VP'): 0.4033185477246591,
 ('B-VP', 'B-ADVP'): 0.15988779803646563,
 ('B-ADVP', 'B-ADJP'): 0.0336950314106225,
 ('B-ADJP', 'I-ADJP'): 0.8536585365853658,
 ('I-ADJP', 'I-ADJP'): 0.14634146341463414,
 ('I-ADJP', 'B-PP'): 0.008919345189536085,
 ('B-PP', 'B-NP'): 0.3607229679737871,
 ('I-NP', 'B-PP'): 0.46467612987436774,
 ('I-NP', 'O'): 0.5198558981233244,
 ('O', 'O'): 0.11352211796246649,
 ('O', 'B-ADJP'): 0.11936036550542548,
 ('B-NP', 'B-VP'): 0.33754996988116753,
 ('B-VP', 'B-PP'): 0.09805841083374123,
 ('I-NP', 'I-NP'): 0.4066787565715961,
 ('O', 'END'): 0.9915176823698291,
 ('B-VP', 'B-SBAR'): 0.2459189046866772,
 ('B-SBAR', 'B-NP'): 0.03502800972413064,
 ('B-VP', 'B-NP'): 0.13326286861853925,
 ('B-VP', 'O'): 0.0515666890080429,
 ('O', 'B-NP'): 0.17520346686396787,
 ('I-NP', 'B-NP'): 0.05498361695381038,
 ('B-ADVP', 'B-PP'): 0.033066840702670366,
 ('B-VP', 'I-VP'): 0.6721133969878925,
 ('I-VP

# Viterbi

In [80]:
import numpy as np

def viterbi(unique_word_list,nodes):
    #This is for the starting for viterbi
    store=[]   #store = the storage for scores for all the nodes.
    scorelist=[]
    
    
    
    #This is for the start
    for i in range(len(nodes)):
        emission_score = emission(nodes[i],unique_word_list[0])
        transition_score = transition("START",nodes[i])
        score_at_start = np.log(emission_score)+np.log(transition_score)
        store.append(score_at_start)    
        
    scorelist.append(store)
    store=[]
    score_per_node=[]
    
    #This is for the middle portion for viterbi
    if len(unique_word_list) > 1:
        for j in range(len(unique_word_list) - 1): #for the whole length in sentence
            for k in range(len(nodes)): # for each node
                #score per node = prevnode*emission*transition
                
                for l in range(len(nodes)): # l = iterate thru previous node, k= iterate thru current node, j= iterate thru sentence
                    # This is to calculate the current node scores.
                    prev_node = scorelist[j][l]
                    emission_score = emission(nodes[k],unique_word_list[j+1])
                    curr_node = nodes[k]
                    score_per_node.append(prev_node+np.log(emission_score)+np.log(transition(nodes[l],curr_node)))
                    
                store.append(max(score_per_node)) # found max path
                
#                 max_score = max(score_per_node)
#                 max_index = np.argmax(score_per_node)                
#                 label_max = labels[max_index]
                score_per_node=[]
            
            #print(store)
            scorelist.append(store) # store the scores for nodes
            store=[]

                      
        score_at_stop=[]
        #This is for the stop for viterbi
        for m in range(len(nodes)):
            score_at_stop.append(np.log(transition(nodes[m],"END"))+ (scorelist[len(unique_word_list)-1][m])) #at stop.
        scorelist.append(max(score_at_stop))
     
    return scorelist

    
def emission(node,word=""):
    global emission_dict
    # Will use this to search the emission score for the given word
    #emission_dict={"Athe":0.9,"Bthe":0.1,"Adog":0.1,"Bdog":0.9,"Astop":0}   # takes out from dictionary
    pair = word+" "+node
    if pair not in emission_dict.keys():
        replaced_text = "#UNK# "+ node
        if replaced_text in emission_dict.keys():
            score = emission_dict[replaced_text]
        else:
            score = 0 # our train data does not have this node
    else:
        score = emission_dict[pair]
    
    return score

def transition(x1,x2):
    global transition_dic
    #will use this to search the transition from x1 to x2
    pair = x1,x2
    if pair not in transition_dic.keys():
        score = 0
    else:
        score = transition_dic[x1,x2]
    return score







In [84]:
# Running the code: required transition, emission dictionaries, nodes and word input.
#transition_dic = {("START","A"):1,("START","B"):0,("START","END"):0,("A","A"):0.5,("A","B"):0.5,("A","END"):0,("B","A"):0,("B","B"):0.8,("B","END"):0.2}

#node_list = ["A","B"]
# nodes(node)
nodes = node
unl=lines[0]
log_scores = viterbi(unl,nodes) #to prevent underflow problem

#emission_dict={"Athe":0.9,"Bthe":0.1,"Adog":0.1,"Bdog":0.9}
#unl=["the","dog","the"]
for v in log_scores:
    print(np.exp(v))
#viterbi(unl)

[[-6.111206773407506, -4.924476023563249, -inf, -2.0163834057126993, -2.716349003916905, -4.748871821028363, -9.085123305651745, -inf, -8.846261538458682, -inf, -7.4009761120922395, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -9.144394428150616], [-9.055073898457792, -9.56414855262662, -inf, -10.824305597242102, -13.247037793187411, -16.01271810085808, -12.641856780001175, -inf, -13.66702058900036, -inf, -8.386211366940504, -7.7392582523555955, -7.575512012951604, -inf, -3.6056186108292803, -11.583395584691102, -inf, -inf, -inf, -10.06122962449076, -12.633823054125497], [-11.142613798255564, -12.99573628993642, -inf, -12.012032685087304, -16.73646641916229, -21.16601428967988, -16.699161091555776, -inf, -16.23943524222807, -inf, -15.786275817206164, -10.683125377405881, -11.992828386267028, -inf, -6.111144547820016, -14.45761505495086, -inf, -inf, -inf, -18.009654865209264, -20.73840506620308], [-18.067297826626273, -21.600723000370177, -inf, -19.412097135352962, -21.03522578

C:\Users\cooll\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in log
  del sys.path[0]
C:\Users\cooll\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: RuntimeWarning: divide by zero encountered in log
C:\Users\cooll\Anaconda3\lib\site-packages\ipykernel_launcher.py:40: RuntimeWarning: divide by zero encountered in log


In [73]:
emission_dict

{'! O': 0.0006283512064343163,
 '# B-ADJP': 0.0017133066818960593,
 '# B-NP': 0.00042278828876440123,
 '# I-NP': 0.00018318037771793887,
 '# O': 4.1890080428954424e-05,
 '#UNK# B-ADJP': 0.1553398058252427,
 '#UNK# B-ADVP': 0.06227208976157083,
 '#UNK# B-INTJ': 0.34615384615384615,
 '#UNK# B-LST': 0.09090909090909091,
 '#UNK# B-NP': 0.0825071345523729,
 '#UNK# B-PP': 0.0025017675531625607,
 '#UNK# B-SBAR': 0.001579778830963665,
 '#UNK# B-VP': 0.07798039537812824,
 '#UNK# I-ADJP': 0.22996515679442509,
 '#UNK# I-ADVP': 0.08264462809917356,
 '#UNK# I-INTJ': 0.2857142857142857,
 '#UNK# I-NP': 0.13883240827242585,
 '#UNK# I-UCP': 0.25,
 '#UNK# I-VP': 0.10404567378679004,
 '#UNK# O': 0.0023458445040214475,
 '$ B-ADJP': 0.0057110222729868645,
 '$ B-NP': 0.022957404079906988,
 '$ I-NP': 0.006924218277738089,
 '$ O': 8.378016085790885e-05,
 '% I-ADJP': 0.027874564459930314,
 '% I-ADVP': 0.0027548209366391185,
 '% I-NP': 0.018556172262827207,
 '& I-NP': 0.0030774303456613727,
 '& O': 0.0001256702

In [20]:
def preprocess_training_blank_row(data):
    start = time.process_time()   
    
    df= pd.read_csv(data, sep='/n', delimiter=None, names=['original'],index_col=False,engine="python",skip_blank_lines=False)
    # dropping null value columns to avoid errors 
    
    # new data frame with split value columns 
    df["x"], df["y"] = split_into_columns(df["original"])
    return df

In [21]:
predata_blank=preprocess_training_blank_row(train_data)
predata_blank

,original,x,y
0,Municipal B-NP,Municipal,B-NP
1,bonds I-NP,bonds,I-NP
2,are B-VP,are,B-VP
3,generally B-ADVP,generally,B-ADVP
4,a B-ADJP,a,B-ADJP
5,bit I-ADJP,bit,I-ADJP
6,safer I-ADJP,safer,I-ADJP
7,than B-PP,than,B-PP
8,corporate B-NP,corporate,B-NP
9,bonds I-NP,bonds,I-NP


In [23]:
lines=[]
line=[]
x= predata_blank
for label in x['x']:
    if pd.isnull(label)==False:
        line.append(label)
    else:
#         line += ' stop'
        lines.append(line)
        line = []

print(lines)

[['Municipal', 'bonds', 'are', 'generally', 'a', 'bit', 'safer', 'than', 'corporate', 'bonds', 'in', 'a', 'recession', ',', 'but', 'not', 'as', 'safe', 'as', 'bonds', 'issued', 'by', 'the', 'federal', 'government', '.'], ['He', 'added', 'that', 'the', 'cost', 'for', 'stress-related', 'compensation', 'claims', 'is', 'about', 'twice', 'the', 'average', 'for', 'all', 'injury', 'claims', '.'], ['The', 'incident', 'occurred', 'Saturday', 'night', '.'], ['He', 'said', ':', '``', 'We', 'know', 'of', 'no', 'technology', 'that', 'exists', 'anywhere', 'in', 'the', 'world', 'that', 'would', 'allow', 'us', 'to', "''", 'reinforce', 'the', 'columns', '.'], ['Walter', 'H.', 'Monteith', 'Jr.', ',', 'SNET', 'chairman', 'and', 'chief', 'executive', 'officer', ',', 'said', ':', '``', 'Innovative', 'marketing', 'of', 'our', 'products', 'and', 'services', 'contributed', 'to', 'increase', 'revenue', '.', "''"], ['The', 'defense', 'lawyers', ',', 'three', 'of', 'whom', 'are', 'solo', 'practitioners', ',', 's

In [39]:
node = np.unique(predata['y'].astype(str))
print(node)

['B-ADJP' 'B-ADVP' 'B-CONJP' 'B-INTJ' 'B-LST' 'B-NP' 'B-PP' 'B-PRT'
 'B-SBAR' 'B-UCP' 'B-VP' 'I-ADJP' 'I-ADVP' 'I-CONJP' 'I-INTJ' 'I-NP'
 'I-PP' 'I-SBAR' 'I-UCP' 'I-VP' 'O']


In [61]:
transition("START","B-ADJP")

0.014277555682467162